In [2]:
# !pip3 install -r requirements.txt
# !pip install ipywidgets

# ## If widgets misbehave, run below commands and restart jupyter

# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user
# !jupyter nbextension enable varInspector/main

### Postgres AACT login
variables.py file lists required entities for trial_to_paper_utils functions in this notebook.

One of note is to fill in your username and password for your AACT account

In [2]:
import ipywidgets as widgets
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from trial_to_paper_utils import *
from pprint import pprint

### Notes on TODOs

- [] % trials with associated papers at all
- [] What AEs are unique, if any, to the papers (not mentioned in trial)
- [] How do authors prioritise which AEs they are discussing in paper? Are there any trends in this?
- [] What is the lag time between trial completion and papers being published?
- [] Extract target - drug - AEs
- [] Is there any other data in CT.gov which could help inform on AE prioritisation
- [] Can we define 'severity' of AEs with models - check w Ines for severity score model
- [x] Compare examples from OTs paper (less severe indications) to cancer examples, intention to link to Ines' severity score
- [x] Sort the AE hits so they read as intended based on their indices

In [7]:
nct_id = "NCT01753193" # 'Easy' SLE example - only 1 patient group
# nct_id = "NCT05034952" # Efficacy and Safety VX-548

# Nav18_17 trials v results
# NCT02215252 NCT01769274 NCT01529346
# nct_id = "NCT01769274" #x

# Diabetes
# nct_id = "NCT03084965"

query = f"""
SELECT 
	c.names AS indication,
    s.{nct_id},
    s.brief_title AS study_title,
    s.phase AS study_phase,
	ae.adverse_event_term as adverse_event,
    ae.event_type,
	ae.organ_system,
	ae.subjects_at_risk,
    ae.subjects_affected as subjects_affected,
	ae.ctgov_group_code,
    -- ae.description,
	ae.result_group_id
FROM 
    "reported_events" as ae
JOIN
	all_conditions as c on c.nct_id = ae.nct_id
JOIN
    studies as s ON s.nct_id = ae.nct_id
WHERE
	ae.nct_id = '{nct_id}';
"""

## AACT queried with starting NCT ID of interest to collect referenced adverse events

In [9]:
study_title, aes, severe_aes, other_aes, patient_groups = aact_data_gather(nct_id, query)
# TODO Get % affected / see relevance of 'other' vs 'serious'
if study_title:
    print(
        f"For CT.gov trial:\n\t'{study_title}' ({nct_id}),\n{len(aes)} unique AEs were recorded in {len(patient_groups)} patient group(s)\n")
    # [print(f"- {a}\n") for a in aes]
else:
    print(f"No trials were found searching for ID: {nct_id}")

UndefinedColumn: column s.nct01753193 does not exist
LINE 4:     s.NCT01753193,
            ^

In [13]:
start_statement = f'Adverse events recorded for trial: {nct_id}'
display_widget(text=aes, placeholder='AEs', start_statement=start_statement)

NameError: name 'widgets' is not defined

## Search ePMC for papers mentioning trial ID, failing this search for papers relating to compound name

In [108]:
trial_in_pmids = query_epmc(query=nct_id, page_size=25)
print(f"NCT ID referenced in {len(trial_in_pmids)} PubMed paper(s).\n{trial_in_pmids}")

if len(trial_in_pmids) > 0:
    test_pmid = trial_in_pmids[0] #TODO - remove test, do for all results & consider subject test groups
    print(test_pmid)
    text = query_bioc(pmid=test_pmid)
else:
    pass
print_text = False
if print_text:
    print("\n".join(text))
    print(len(text))
    print(text[3])

NCT ID referenced in 6 PubMed paper(s).
['39193183', '37148484', '31190735', '33225631', '34768756', '35383948']
39193183


## Testing HuggingFace AE models over literature text

In [109]:
from typing import List


def check_ahead(i: int, input_df: pd.DataFrame, df_set: List, entity_str: str) -> [List, str]:
    try:
        if input_df.iloc[i+1].entity == 'B-AE': # Next term is a new phrase
            df_set.append(entity_str)
        else:
            while input_df.iloc[i+1].entity == 'I-AE':
                if input_df.iloc[i+1].word.startswith('##'): # Clean up hashes
                    word = input_df.iloc[i+1].word
                    entity_str = entity_str + word[2:]
                else:
                    entity_str = entity_str + ' ' + input_df.iloc[i+1].word
                i += 1
    except IndexError as e:
        df_set.append(entity_str) # End of df
    return df_set, entity_str

def get_set_from_col(input_df: pd.DataFrame) -> List:
    input_df = input_df.sort_values('start')
    df_set = []
    entity_str = ''
    for i, row in input_df.iterrows():
        if row.entity == 'B-AE':
            # Assign start of term
            entity_str = row.word
            # Look ahead of additional words of term
            df_set, entity_str = check_ahead(i, input_df, df_set, entity_str)

    # return list(set(df_set))
    return df_set

In [113]:
# TODO Test open source models for AE detection in text, compare to those recorded in trial
pipe = pipeline(task="token-classification", model="MutazYoune/BiomedBERT-Adverse-Events-NER_pun", tokenizer="MutazYoune/BiomedBERT-Adverse-Events-NER_pun")
# pipe = pipeline(task="token-classification", model="MutazYoune/Medical-NER-Adverse-Events-NER", tokenizer="MutazYoune/Medical-NER-Adverse-Events-NER")

all_sections = []
all_aes = []
for x in tqdm(text):
    try:
        res = pipe(x)
        if res:
            section_df = pd.DataFrame(res)
            section_set = get_set_from_col(section_df)
            [all_aes.append(x) for x in section_set]
            pprint(section_df)
            all_sections.append(section_df)
    except:
        continue

paper_aes_df = pd.concat(all_sections, ignore_index=True)
model_name = pipe.model.name_or_path
model_name = model_name.rpartition('/')[-1]
# paper_aes.to_csv(f'./output/{model_name}_AEs_{test_pmid}.csv')

Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
  2%|█▉                                                                                  | 5/215 [00:00<00:12, 16.34it/s]

  entity     score  index word  start  end
0   I-AE  0.580194     22  ##s    122  123


 11%|█████████▎                                                                         | 24/215 [00:00<00:05, 33.59it/s]

  entity     score  index       word  start  end
0   B-AE  0.681990      6  increased     34   43
1   I-AE  0.813916      7     levels     44   50
2   I-AE  0.757584      8         of     51   53
3   I-AE  0.537050      9        ifn     54   57
4   I-AE  0.743058     10         in     58   60
5   I-AE  0.565262     11      serum     61   66


 15%|████████████▋                                                                      | 33/215 [00:01<00:05, 31.79it/s]

...

 78%|████████████████████████████████████████████████████████████████                  | 168/215 [00:04<00:01, 41.22it/s]

  entity     score  index       word  start  end
0   B-AE  0.636648      1  photosens      0    9
1   I-AE  0.577762      2  ##itivity      9   16
  entity     score  index       word  start  end
0   I-AE  0.620246     14  nephritis     94  103


 86%|██████████████████████████████████████████████████████████████████████▌           | 185/215 [00:04<00:00, 43.07it/s]

  entity     score  index   word  start  end
0   I-AE  0.512464      7  rates     26   31


100%|██████████████████████████████████████████████████████████████████████████████████| 215/215 [00:05<00:00, 38.61it/s]


In [111]:
all_aes

['increased levels of ifn in serum',
 'herpes zoster incidence',
 'headaches',
 'injection site induration',
 'photosensitivity']

In [112]:
from IPython.display import display, HTML

def make_scrollable(df, height=500):
    return HTML(f"<div style='max-height:{height}px; overflow:auto'>{df.to_html()}</div>")

paper_aes_df = paper_aes_df.sort_values('start')
make_scrollable(paper_aes_df)

,entity,score,index,word,start,end
29,B-AE,0.636648,1,photosens,0,9
30,I-AE,0.577762,2,##itivity,9,16
27,I-AE,0.532042,3,event,16,21
32,I-AE,0.512464,7,rates,26,31
1,B-AE,0.681990,6,increased,34,43
2,I-AE,0.813916,7,levels,44,50
3,I-AE,0.757584,8,of,51,53
4,I-AE,0.537050,9,ifn,54,57
7,I-AE,0.583950,12,overall,57,64
5,I-AE,0.743058,10,in,58,60


In [96]:
print(f'{len(list(set(aes)))} recorded in clinical trial, NCT ID: {nct_id}')

print(f'{len(all_aes)} detected in paper, PMID: {test_pmid}')

339 recorded in clinical trial, NCT ID: NCT01769274
27 detected in paper, PMID: 26941184


### Novelty / inaccuracy of trial AEs

In [103]:
unique_aes_to_paper = set(all_aes).difference(set(aes))
unique_aes_to_paper

{'acute',
 'acute pain pain cold allodynia',
 'anosmia',
 'atrial fibrillation',
 'congenital to',
 'enhanced cold pain',
 'episodic',
 'erythromelalgia',
 'extreme',
 'familial',
 'heat',
 'heat hyperalgesia',
 'hyperalgesia',
 'inflammatory pain',
 'insensitivity to pain',
 'loss of perception',
 'moderate to severe inadequately controlled cancer - related pain',
 'neuropathic orofacial pain',
 'neuropathic pain',
 'orofacial neuropathic pain',
 'pain',
 'pain hypersensitivity',
 'painful neuropathytaity',
 'painruciating mechanically evoked pain',
 'postetic',
 'trigeminal'}

In [104]:
unique_aes_to_trial = set(aes).difference(set(all_aes))
len(unique_aes_to_trial)

339

### Trying w OTs reasons for trial stoppage texts

Texts not specific enough in most cases

In [42]:
ots_trial_stop = pd.read_json("hf://datasets/opentargets/clinical_trial_reason_to_stop/data.json", lines=True)
ots_trial_stop_safety = ots_trial_stop[ots_trial_stop["label_descriptions"].apply(lambda x: "Safety_Sideeffects" in x)]
text = list(ots_trial_stop_safety["text"])

display_widget(text=text, placeholder='Trial stoppage AEs', start_statement='OTs, reasons for trial stoppage')

OTs, reasons for trial stoppage


In [36]:
# pipe = pipeline(task="token-classification", model="MutazYoune/BiomedBERT-Adverse-Events-NER_pun", tokenizer="MutazYoune/BiomedBERT-Adverse-Events-NER_pun")
pipe = pipeline(task="token-classification", model="MutazYoune/Medical-NER-Adverse-Events-NER", tokenizer="MutazYoune/Medical-NER-Adverse-Events-NER")
all_sections = []
for x in tqdm(text):
    try:
        # print(x)
        res = pipe(x)
        if res:
            # print([x['word'] for x in res])
            # print('\n')
            section_df = pd.DataFrame(res)
            # pprint(section_df)
            all_sections.append(section_df)
    except:
        continue
ots_trial_stoppage_aes = pd.concat(all_sections, ignore_index=True)

Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1. Very low enrollment rate. Ê Ê 2. Recent studies question the effect of adding panitumumab in this category of patients. Ê Ê 3. Too high toxicity rate


  1%|▍                                          | 2/211 [00:00<00:49,  4.26it/s]

2 complications with midazolam
5 immune related serious adverse events in Phase 1 study


  1%|▌                                          | 3/211 [00:00<00:47,  4.36it/s]

...

four adverse events encountered
higher incidence of pain related sexual dysfunction in the TO arm
['▁pain', '▁related', '▁sexual', '▁dysfunction']


investigation of adverse events
negative outcome
post-surgical complications in the last 2 patients
potential harm of insulin infusion outweights the benefit.
reaching of step 1 (recruitment of 8 patients) per protocol -&gt; risk assessment -&gt; termination because of occurance of toxicity Grade 3 and 4


 98%|████████████████████████████████████████▏| 207/211 [00:15<00:00, 19.84it/s]

safety concerns regarding use of rosiglitazone
safety issues (toxicity)
safety reasons
side effect profile did not match expectations
toxicities required dose reduction compromising effectiveness
toxicity


100%|█████████████████████████████████████████| 211/211 [00:15<00:00, 13.57it/s]

unexpected level of toxicity
voluntarily by Sponsor to investigate an anticipated SAE


In [32]:
pd.set_option('display.max_columns', None)
ots_trial_stoppage_aes

,entity,score,index,word,start,end
0,B-AE,0.977275,2,immune,2,8
1,I-AE,0.981264,3,related,9,16
2,I-AE,0.986399,4,serious,17,24
3,I-AE,0.994388,5,adverse,25,32
4,I-AE,0.994540,6,events,33,39
...,...,...,...,...,...,...
129,B-AE,0.833021,4,pain,20,24
130,I-AE,0.876938,5,related,25,32
131,I-AE,0.484166,6,sexual,33,39
132,I-AE,0.989740,7,dysfunction,40,51


### ADE corpus

In [49]:
df = pd.read_parquet("hf://datasets/ade-benchmark-corpus/ade_corpus_v2/Ade_corpus_v2_drug_ade_relation/train-00000-of-00001.parquet")
text = list(df["text"])

# pipe = pipeline(task="token-classification", model="MutazYoune/BiomedBERT-Adverse-Events-NER_pun", tokenizer="MutazYoune/BiomedBERT-Adverse-Events-NER_pun")
pipe = pipeline(task="token-classification", model="MutazYoune/Medical-NER-Adverse-Events-NER", tokenizer="MutazYoune/Medical-NER-Adverse-Events-NER")
all_sections = []
for x in tqdm(text):
    try:
        print(x)
        res = pipe(x)
        if res:
            print([x['word'] for x in res])
            print('\n')
            section_df = pd.DataFrame(res)
            # pprint(section_df)
            all_sections.append(section_df)
    except:
        continue
ade_bench = pd.concat(all_sections, ignore_index=True)

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=None)"), '(Request ID: 88e37dda-af95-4a31-aa4f-697ac6c0ec69)')

In [47]:
df = pd.read_parquet("hf://datasets/ade-benchmark-corpus/ade_corpus_v2/Ade_corpus_v2_drug_ade_relation/train-00000-of-00001.parquet")
df

,text,drug,effect,indexes
0,Intravenous azithromycin-induced ototoxicity.,azithromycin,ototoxicity,"{'drug': {'start_char': [12], 'end_char': [24]..."
1,"Immobilization, while Paget's bone disease was...",dihydrotachysterol,increased calcium-release,"{'drug': {'start_char': [91], 'end_char': [109..."
2,Unaccountable severe hypercalcemia in a patien...,dihydrotachysterol,hypercalcemia,"{'drug': {'start_char': [84], 'end_char': [102..."
3,METHODS: We report two cases of pseudoporphyri...,naproxen,pseudoporphyria,"{'drug': {'start_char': [58], 'end_char': [66]..."
4,METHODS: We report two cases of pseudoporphyri...,oxaprozin,pseudoporphyria,"{'drug': {'start_char': [71], 'end_char': [80]..."
...,...,...,...,...
6816,Lithium treatment was terminated in 1975 becau...,Lithium,lithium intoxication,"{'drug': {'start_char': [0], 'end_char': [7]},..."
6817,Lithium treatment was terminated in 1975 becau...,lithium,lithium intoxication,"{'drug': {'start_char': [52], 'end_char': [59]..."
6818,Eosinophilia caused by clozapine was observed ...,clozapine,Eosinophilia,"{'drug': {'start_char': [23], 'end_char': [32]..."
6819,Eosinophilia has been encountered from 0.2 to ...,clozapine,Eosinophilia,"{'drug': {'start_char': [55], 'end_char': [64]..."


## Early phase kidney failure trials

In [5]:
nct_id = "NCT06901102"
nct_id = "NCT06901011"
query = f"""
SELECT 
	c.names AS indication,
    s.{nct_id},
    s.brief_title AS study_title,
    s.phase AS study_phase,
	ae.adverse_event_term as adverse_event,
    ae.event_type,
	ae.organ_system,
	ae.subjects_at_risk,
    ae.subjects_affected as subjects_affected,
	ae.ctgov_group_code,
    -- ae.description,
	ae.result_group_id
FROM 
    "reported_events" as ae
JOIN
	all_conditions as c on c.nct_id = ae.nct_id
JOIN
    studies as s ON s.nct_id = ae.nct_id
WHERE
	ae.nct_id = '{nct_id}';
"""

In [6]:
study_title, aes, severe_aes, other_aes, patient_groups = aact_data_gather(nct_id, query)
# TODO Get % affected / see relevance of 'other' vs 'serious'
if study_title:
    print(
        f"For CT.gov trial:\n\t'{study_title}' ({nct_id}),\n{len(aes)} unique AEs were recorded in {len(patient_groups)} patient group(s)\n")
    # [print(f"- {a}\n") for a in aes]
else:
    print(f"No trials were found searching for ID: {nct_id}")

UndefinedColumn: column s.nct06901011 does not exist
LINE 4:     s.NCT06901011,
            ^